# Classes/Functions to record data while watching a baseball game!

In [1]:
# Import statements
import pandas as pd

### Game Recorder

In [2]:
class Game_Recorder:
    '''
    Class allowing user to record play-by-play during a game. Assumes starting at first plate 
    appearance of game, unless initial_state dict is provided to describe a later starting point. 
    See example format for initial_state dict below.
    
    Records data for both teams in single pandas dataframe.
    
    >>> PA_Recorder(initial_state = { 
                    'Pitcher':['Michael Wacha'], 'Batter':['Thairo Estrada'], 'Inning':[3], 
                    'Half-Inning':['Top'], Outs':[1], 'Strikes':[2], 'Balls':[1], 
                   'First':[True], 'Second':[False], 'Third':[False]
                                   }
                   )
    
    '''
    def __init__(self, home_lineup: list = None, away_lineup: list = None, initial_state: dict = None):
        '''
        Begins game log, starting at first PA or PA described in initial_state, if provided.
        Prompts user for lineup info if not provided. Refer to code for format of inputs.
        '''
        # Class Import Statements
        import pandas as pd
        
        self.home_team = input("What is the home team?")
        self.away_team = input("What is the away team?")
        if not initial_state: # Prompt user for some extra info on the game
            pitcher = input(f"What is the name of the {self.home_team}'s pitcher?")
            batter = input(f"Who is leading off for the {self.away_team}?")
            initial_state = {'Pitcher':[pitcher], 'Batter':[batter], 'Inning':[1], 'Half-Inning':['Top'], 
                             'Outs':[0], 'Strikes':[0], 'Balls':[0], 
                             'First':[False], 'Second':[False], 'Third':[False]
                            }
        
        self.pitcher = initial_state["Pitcher"][-1]
        pitcher_hand = input(f"What hand does {self.pitcher} throw with? ('L'/'R')")
        away_pitcher = input(f"Who is pitching for the {self.away_team}?")
        away_pitcher_hand = input(f"What hand does {away_pitcher} throw with?")
            
        if not home_lineup: # Prompt user for home team's lineup info
            home_lineup = self.lineup(self.home_team)
                
        if not away_lineup: # Prompt user for away team's lineup info
            away_lineup = self.lineup(self.away_team)
        
        # Declare all these beautiful instance variables for fine tracking
        self.game_log = initial_state
        self.pitcher = initial_state["Pitcher"][0] # Redefining just for consistent visibility
        self.batter = initial_state["Batter"][0]
        self.strikes = initial_state["Strikes"][0]
        self.balls = initial_state["Balls"][0]
        self.outs = initial_state["Outs"][0]
        self.inning = initial_state["Inning"][0]
        self.half_inning = initial_state["Half-Inning"][0]
        self.first = initial_state["First"][0]
        self.second = initial_state["Second"][0]
        self.third = initial_state["Third"][0]
        self.home_pitchers = [{"Name":self.pitcher, "Hand":pitcher_hand, "Team":self.home_team}]
        self.away_pitchers = [{"Name":away_pitcher, "Hand":away_pitcher_hand, "Team":self.away_team}]
        self.home_lineup = home_lineup
        self.away_lineup = away_lineup
        
        
        
        
    def lineup(self, team_name:str) -> list:
        '''
        Prompts user for lineup info for a given team. Returns info as a list of batter info dicts.
        '''
        lineup_list = []
        
        for i in range(1,10):
            batter = input(f"Please enter the name of the {i}-hole batter for the {team_name} (titlecase, no accent marks).")
            batter_hand = input(f"\n Which side does {batter} bat from? Enter 'L' or 'R'. \n")
            batter_team = team_name
            lineup_list.append({"Name":batter, "Hand":batter_hand, "Team":team_name, "Spot":i})
                
        return(lineup_list)
        
        

                           
    def next_batter(self):
        '''
        Updates batter variable to next batter.
        '''
        # Fresh count
        self.strikes = 0
        self.balls = 0
        
        if self.half_inning == 'Bottom': # Home team batting
            if self.inning != 1: # Don't need to rotate home lineup in 1st inning!
                last_batter = self.home_lineup.pop(0)
                self.home_lineup.append(last_batter)
                self.batter = self.home_lineup[0]["Name"]
        else: # Away team batting
            last_batter = self.away_lineup.pop(0)
            self.away_lineup.append(last_batter)
            self.batter = self.away_lineup[0]["Name"]
            
                           
                           
                           
    def foul(self):
        '''
        Records a foul ball and updates strikes appropriately.
        '''
        if self.strikes < 2: # Only adds a strike if less than 2 strikes
            self.strikes += 1
                           
                           
                           
                           
    def strikeout(self):
        '''
        Records a strikeout and updates related variables appropriately.
        '''
        self.outs += 1
        self.strikes = 0
        self.balls = 0
        if self.outs < 3:
            self.next_batter()
        else: # Inning Change!
            self.inning_change()
            
            
            
    def walk(self):
        '''
        Records a walk and updates related variables appropriately.
        '''
        if self.first: # Man on first
            if self.second: # Men on first and second
                self.third = True
            else:
                self.second = True
        else: # No baserunners forced ahead
            self.first = True
            
        # Clean count for next batter
        self.strikes = 0
        self.balls = 0
        self.next_batter()
                          
            
            
            
    def pinch_hit(self):
        '''
        Pinch hit for a batter and update related variables.
        '''
        # Check which team is batting
        team = self.home_team
        if self.half_inning == 'Top': # Away team batting
            team = self.away_team
        
        # Ask for new batter's name and handedness, replace in lineup, update self.batter
        new_batter = input(f"Who is the new batter for the {team}?")
        new_batter_hand = input(f"What hand does {new_batter} bat with? ('L'/'R')")
        if team == self.home_team:
            spot = self.home_lineup[0]["Spot"]
            self.home_lineup[0] = {"Name":new_batter, "Hand":new_batter_hand, "Team":self.home_team, "Spot":spot}
            
        else: # Away team
            spot = self.away_lineup[0]["Spot"]
            self.away_lineup[0] = {"Name":new_batter, "Hand":new_batter_hand, "Team":self.away_team, "Spot":spot}
            
        self.batter = new_batter
        self.game_log["Batter"][-1] = self.batter
        
        
        
        
    def hit_by_pitch(self):
        '''
        Records a hit-by-pitch and updates related variables appropriately.
        '''
        if self.balls != 0: # If 0 balls, HBP happened on 4th ball and triggered the walk already
            self.walk()
            self.refresh_log()
            
        self.game_log["Play Result"][-1] = 'HBP'
        
        
        
        
    def wild_pitch(self):
        '''
        Records a wild pitch and adjusts related variables accordingly.
        '''
        if self.game_log["Play Result"][-1] == 'K': # Last pitch induced a strikeout
            
            if self.outs != 0: # Inning didn't change, only need to take away an out
                self.outs -= 1
                
            else: # 0 outs means last pitch ended previous half-inning; need to revert
                self.outs = 2
                
                if self.half_inning == 'Top': # Go back to bottom of previous inning
                    self.inning -= 1
                    self.half_inning = 'Bottom'
                    self.batter = self.home_lineup[0]["Name"]
                    self.pitcher = self.away_pitchers[-1]["Name"]
                    
                else: # Go back to top of current inning
                    self.half_inning = 'Top'
                    self.batter = self.away_lineup[0]["Name"]
                    self.pitcher = self.home_pitchers[-1]["Name"]
        
        # Check baserunners
        self.first = bool(int(input("Enter 1 if a man is now on first, or 0 if not.")))
        self.second = bool(int(input("Enter 1 if a man is now on second, or 0 if not.")))
        self.third = bool(int(input("Enter 1 if a man is now on third, or 0 if not.")))
        
        # Note the wild pitch alongside the previous play result
        self.game_log["Play Result"][-1] += ' (WP)'
        
        # Adjust game log
        self.refresh_log()
                
        
        
        
    def passed_ball(self):
        '''
        Records a passed ball and adjusts related variables accordingly.
        '''
        if self.game_log["Play Result"][-1] == 'K': # Last pitch induced a strikeout
            
            if self.outs != 0: # Inning didn't change, only need to take away an out
                self.outs -= 1
                
            else: # 0 outs means last pitch ended previous half-inning; need to revert
                self.outs = 2
                
                if self.half_inning == 'Top': # Go back to bottom of previous inning
                    self.inning -= 1
                    self.half_inning = 'Bottom'
                    self.batter = self.home_lineup[0]["Name"]
                    self.pitcher = self.away_pitchers[-1]["Name"]
                    
                else: # Go back to top of current inning
                    self.half_inning = 'Top'
                    self.batter = self.away_lineup[0]["Name"]
                    self.pitcher = self.home_pitchers[-1]["Name"]
        
        # Check baserunners
        self.first = bool(int(input("Enter 1 if a man is now on first, or 0 if not.")))
        self.second = bool(int(input("Enter 1 if a man is now on second, or 0 if not.")))
        self.third = bool(int(input("Enter 1 if a man is now on third, or 0 if not.")))
        
        # Note the passed ball alongside the previous play result
        self.game_log["Play Result"][-1] += ' (PB)'
        
        # Adjust game log
        self.refresh_log()
        
        
    def steal(self):
        '''
        Records info for a steal attempt (failed or successful) and updates variables accordingly.
        Employs a counter system to track occupied bases.
        '''
        # Ask for what base(s) the runner(s) attepted to steal, and did the runner(s) succeed
        bases = input("Enter the number(s) for the base(s) the runner(s) attempted to steal, separated by commas. (2-4, 2nd-Home)")
        successes = input("Enter 1('s) if the steal(s) was/were successful, 0('s) if not, separated by commas.")
        
        # Convert inputs to lists of ints and bools, respectively
        bases = [int(base.strip()) for base in bases.split(',')]
        successes = [bool(int(success.strip())) for success in successes.split(',')]
                
        # Conveniently, changing the base attribute from bool to int gives its starting baserunner count :)
        first = int(self.first)
        second = int(self.second)
        third = int(self.third)
        
        # Loop through steal attempts, update base counters, record outs as needed
        n = len(bases)
        for i in range(n):

            if not successes[i]: # Caught stealing :(
                self.outs += 1
                self.game_log["Play Result"][-1] += ' (CS)'
                
            else: # Stolen base!
                if bases[i] == 2:
                    second += 1
                elif bases[i] == 3:
                    third += 1
                #else:
                    # Run scores, but not tracking runs right now
                self.game_log["Play Result"][-1] += ' (SB)'
                    
            # Regardless of success, steal attempt vacates base runner was on
            vacated_base = bases[i] - 1
            if vacated_base == 1:
                first -= 1
            elif vacated_base == 2:
                second -= 1
            else: # Oof, out trying to steal home
                third -= 1
                
        # Base counters should all be 0 or 1, so convert back to bool to update base attribute
        self.first = bool(first)
        self.second = bool(second)
        self.third = bool(third)
        
        # If there are now 3 outs, perform an inning change BUT make sure batter leads off next inning
        if self.outs >= 3:
            
            # Cycle through lineup so current batter comes to plate next inning
            for i in range(8):
                self.next_batter()
                
            # Now perform inning change
            self.inning_change()
                
        # Adjust game log
        self.refresh_log()
    
    
    
    
    def balk(self):
        '''Records a balk and advances runners appropriately.'''
        if self.first: # First
            if self.second: # First and Second
                self.third = True
            elif self.third: # First and Third
                self.third = False
            self.second = True
            self.first = False
        elif self.second: # Second
            self.third = True
            self.second = False
        else: # Third
            self.third = False
            
        self.game_log["Play Result"][-1] += ' (Balk)'
        self.refresh_log()
    
        
        
        
    def error(self):
        '''
        Catch-all error function. Prompts user for resulting game state and updates related variables.
        '''        
        # Check baserunners
        self.first = bool(int(input("Enter 1 if a man is now on first, or 0 if not.")))
        self.second = bool(int(input("Enter 1 if a man is now on second, or 0 if not.")))
        self.third = bool(int(input("Enter 1 if a man is now on third, or 0 if not.")))
        
        # Check outs, balls, strikes
        self.outs = int(input("Enter the number of outs."))
        self.balls = int(input("Enter the number of balls."))
        self.strikes = int(input("Enter the number of outs."))
        
        # Check inning
        self.inning = input("What inning is it? (Just the number, please.)")
        self.half_inning = input("Is it the 'Top' or 'Bottom' of the inning?")
        
        # Adjust game log
        self.game_log["Play Result"][-1] += ' (Error)'
        self.refresh_log()
        
                           
     
    
    def inning_change(self):
        '''
        Records an inning change and updates related variables appropriately.
        '''
        # Clean slate
        self.strikes = 0
        self.balls = 0
        self.outs = 0
        self.first = False
        self.second = False
        self.third = False
        
        if self.half_inning == 'Top': # Inning number doesn't change
            self.half_inning = 'Bottom'
            
        else: # Going from bottom of current inning to top of next
            self.half_inning = 'Top'
            self.inning += 1
        
        self.next_batter()
        self.pitching_change()
            
            
            
    def pitching_change(self, context:str = 'Inning'):    
        '''
        Records a pitching change with the specified context and updates related variables appropriately.
        Context can be 'Inning' or 'Bullpen'.
        '''
        if context == 'Inning': # No new pitchers, just other team pitching now
            
            if self.half_inning == 'Bottom': # Away team pitches bottom of inning
                self.pitcher = self.away_pitchers[-1]["Name"]
                
            else: # Going into the top of the next inning, home team pitching
                self.pitcher = self.home_pitchers[-1]["Name"]
                
        else: # Bullpen context, need new pitcher's info
            
            team = self.home_team
            if self.half_inning == 'Bottom': # Away team pitching
                team = self.away_team
                
            new_pitcher = input(f"Who is the new pitcher for the {team}?")
            new_pitcher_hand = input(f"What hand does {new_pitcher} throw with? ('L'/'R')")
            if team == self.home_team:
                self.home_pitchers.append({"Name":new_pitcher, "Hand":new_pitcher_hand, "Team":self.home_team})
                
            else: # Away team
                self.away_pitchers.append({"Name":new_pitcher, "Hand":new_pitcher_hand, "Team":self.away_team})
                
            self.pitcher = new_pitcher
        
    def record(self):
        '''
        Creates user input loop to record each pitch in the game log.
        '''
        stop_con = ''
        while stop_con != 'STOP':
            # Ask a bunch of user input
            pitch_thrown = [input("What pitch was just thrown?")]
            location_thrown = [input("Where was the pitch thrown?")]
            swing = [bool(int(input("Enter 1 if the batter swung, or 0 if not.")))]
            
            contact_result = ['No Swing']
            if swing[0]: # Contact possible
                contact_result = [input("Describe the contact result as 'GB', 'FB', 'LD', 'Whiff', or 'Foul'.")]
                if contact_result[0] == 'Foul':
                    self.foul()
                    play_result = contact_result
                elif contact_result[0] == 'Whiff':
                    self.strikes += 1
                    play_result = ['Strike']
                else: # Prompt user for results of ball-in-play
                    play_result = [input("Describe the result of the play as 'Out', 'DP', 'Hit', or 'Error'.")]
                    if play_result[0] == 'Out':
                        self.outs += 1
                    elif play_result[0] == 'DP':
                        self.outs += 2
                           
                    if self.outs >= 3: # Inning Change!
                        self.inning_change()
                    else: # Check baserunners
                        self.first = bool(int(input("Enter 1 if a man is now on first, or 0 if not.")))
                        self.second = bool(int(input("Enter 1 if a man is now on second, or 0 if not.")))
                        self.third = bool(int(input("Enter 1 if a man is now on third, or 0 if not.")))
                        self.next_batter()
                    
            
            else: # No swing
                play_result = [input("Was the pitch called a 'Strike' or a 'Ball'?")]
                if play_result[0] == 'Strike':
                    self.strikes += 1
                else: # Ball
                    self.balls += 1
                    
            # Check balls, strikes
            if self.strikes == 3: # Strikeout!
                play_result[0] = 'K'
                self.strikeout()
                
            elif self.balls == 4: # Walk!
                play_result[0] = 'BB'
                self.walk()
                

            # Add user input to game log
            self.update_log({"Pitch Thrown":pitch_thrown, "Location Thrown":location_thrown,
                             "Swing":swing, "Contact Result":contact_result, "Play Result":play_result,
                             "First":[self.first], "Second":[self.second], "Third":[self.third], "Inning":[self.inning],
                             "Half-Inning":[self.half_inning], "Outs":[self.outs], "Balls":[self.balls], 
                             "Strikes":[self.strikes], "Pitcher":[self.pitcher], "Batter":[self.batter]
                            }
                           )
        
            # Ask user if they want to continue recording
            stop_con = input("Enter 'STOP' to stop recording, or just hit enter to continue.")
        
        
    def refresh_log(self):
        '''
        Refreshes most recent entry in each attribute array to current attribute value.
        '''
        self.game_log["First"][-1] = self.first
        self.game_log["Second"][-1] = self.second
        self.game_log["Third"][-1] = self.third
        self.game_log["Strikes"][-1] = self.strikes
        self.game_log["Balls"][-1] = self.balls
        self.game_log["Outs"][-1] = self.outs
        self.game_log["Inning"][-1] = self.inning
        self.game_log["Half-Inning"][-1] = self.half_inning
        self.game_log["Pitcher"][-1] = self.pitcher
        self.game_log["Batter"][-1] = self.batter
        
    def update_log(self, additions:dict):
        '''
        Wrapper to update game log with information in given dict.
        '''
        for key in additions.keys():
            if key in self.game_log.keys(): # Adding occurrence of already declared observation type
                self.game_log[key] += additions[key]
            else: # Adding occurrence of a new observation type
                self.game_log[key] = additions[key]
                
                
                
                
    def export_log(self) -> pd.DataFrame:
        '''
        Preps and exports game_log to pandas dataframe.
        '''
        n = len(self.game_log["Pitch Thrown"]) # Correct number of rows
        
        # Game log prefills game state row before completing with pitch info, so need to trim for equal row #'s
        for key in self.game_log.keys():
            self.game_log[key] = self.game_log[key][:n] # Cut off extra balls,strikes,outs,etc.
        
        # Convert to pd.DataFrame and return
        return(pd.DataFrame(self.game_log))